# Exploratory Analysis

In this part of the analysis, we aim to explore and gain an understanding of the data before running any machine learning model. 

In [1]:
#import packages we will be using later
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv(r"C:\Users\\Downloads\Winter 2023 Discovery Days\train_ScotiaDSD.csv")
data.head()

,EVENT_MONTH,EVENT_DAY_OF_WEEK,EVENT_TIME,TRANSACTION_ID,FRAUD_FLAG,AVAIL_CRDT,AMOUNT,CREDIT_LIMIT,CARD_NOT_PRESENT,USER_AGENT,...,COUNT_LOW_AMT_PAST_30DAY,COUNT_MANUAL_ENTRY_PAST_30DAY,COUNT_PHONE_ORDER_PAST_30DAY,COUNT_PURCHASE_EXCLUDING_GAS_PAST_30DAY,COUNT_PLANNED_PAST_30DAY,COUNT_SWIPE_PAST_30DAY,COUNT_TRAVEL_AND_ENTERTAINMENT_PAST_30DAY,COUNT_WEEKEND_PAST_30DAY,PREV_M_INFLATION,PREV_M_UNEMP_RATE
0,2,4,14:16,a316accb,1.0,537.10,11.70,29200.0,1,NaN,...,16,39,73,60,13,22,4,14,1.1,5.7
1,2,4,12:51,8352728b,0.0,20371.88,96.35,30700.0,1,Mozilla/5.0 (iPhone; CPU OS 16_0 like Mac OS X...,...,1,50,93,93,0,0,0,0,1.1,5.7
2,2,4,15:26,d50120e1,0.0,15628.17,193.72,19500.0,0,NaN,...,9,3,42,42,2,33,0,12,1.1,5.7
3,2,4,16:19,e96e2139,0.0,12913.98,47.15,18400.0,0,NaN,...,4,7,75,72,12,61,0,20,1.1,5.7
4,2,4,8:15:,32502a99,0.0,26779.35,121.88,29200.0,1,Mozilla/5.0 (iPhone; CPU OS 16_0 like Mac OS X...,...,11,6,41,42,10,30,4,16,1.1,5.7


In [2]:

#output data dimension
print('Data dimension: {} x {}'.format(data.shape[0], data.shape[1]))
print('Number of fraudulent transactions: {}'.format(data['FRAUD_FLAG'].sum()))

Data dimension: 89230 x 175
Number of fraudulent transactions: 2154.0


In this data, each row represent one credit card transaction. There are 89,230 transactions with 175 attributes, with only 2154 fraudulent transactions, thus making the data very unblanced. 

There are a also lot of indicator fields in our data (Flags) and have already been formatted as such.


</br>
</br>
We observe that there are some missing values in the dataset and we should investigate these values. 

In [3]:
data['FRAUD_FLAG'].sum()/data['TRANSACTION_ID'].count() *100 

2.4139863274683404

In [4]:
#find columns with missing values
data.columns[data.isna().any()].tolist()

['USER_AGENT', 'CITY']

Some transactions have no value for the 'USER_AGENT' and 'CITY' columns. Based on the data dictionary, these are the two columns for E-Commerce transactions, so it makes sense that some values are empty - only E-Commerce transactions will have values for these two columns, otherwise the values will be empty. 

Because only E-Commerce transactions will give values to these two columns, these two columns are always empty or non empty at the same time, we can run the numbers to verify this:

In [5]:
NoUser = data[data['USER_AGENT'].isna()]
NoCity = data[data['CITY'].isna()]
NoUserOrCity = NoUser[NoUser['CITY'].isna()]

'There are {} empty records for USER_AGENT, {} empty records for CITY, and {} records when both of them are empty'.format(NoUser.shape[0], NoCity.shape[0], NoUserOrCity.shape[0])

'There are 77322 empty records for USER_AGENT, 77322 empty records for CITY, and 77322 records when both of them are empty'

We see that USER_AGENT and CITY are always empty at the same time. We can then use non-empty USER_AGENT as a ticker to signal E-Commerce transaction

In [6]:
data['E-COMMERCE_FLAG'] = np.select([data['USER_AGENT'].notnull(),
                                              data['USER_AGENT'].isnull()],
                                             ['E-commerce', 'Not E-commerce'])
pd.crosstab(data['E-COMMERCE_FLAG'],data['FRAUD_FLAG'],margins=True, margins_name="Total", normalize='index')

FRAUD_FLAG,0.0,1.0
E-COMMERCE_FLAG,,
E-commerce,0.922405,0.077595
Not E-commerce,0.984092,0.015908
Total,0.975860,0.024140


We can see that fraudulent transactions account for only 1.6% of transactions in non-E-Commerce transactions, but account for 7.8% of transactions in e-commerce transactions. This is a pretty big indicator that e-commerce transactions are related to fraudulent transactions. We will keep this mind as we continue to explore the data.

</br>

Next, We work on observing some of the columns of interest.

For AVAIL_CRDT,	AMOUNT,	CREDIT_LIMIT, we can compare the fraudulent transactions versus non-fradulent transactions using the describe() function and finding the summary statistics. 

In [7]:
fraud = data[data['FRAUD_FLAG'] == 1]
nonfraud = data[data['FRAUD_FLAG'] == 0]

fraud[fraud.columns[5:8]].describe()

,AVAIL_CRDT,AMOUNT,CREDIT_LIMIT
count,2154.000000,2154.000000,2154.000000
mean,10626.419410,217.606263,16088.625812
std,8980.165705,561.158044,10800.129692
min,0.000000,0.970000,100.000000
25%,2346.337500,15.600000,6825.000000
50%,9408.230000,51.670000,15600.000000
75%,16909.425000,175.092500,22500.000000
max,58825.650000,9750.000000,88100.000000


In [8]:
nonfraud[nonfraud.columns[5:8]].describe()

,AVAIL_CRDT,AMOUNT,CREDIT_LIMIT
count,87076.000000,87076.000000,87076.000000
mean,11848.972113,90.313311,16588.999265
std,8706.579445,455.107996,9631.525298
min,0.000000,0.970000,0.000000
25%,4147.650000,8.200000,9600.000000
50%,11353.900000,25.620000,17500.000000
75%,17896.215000,65.600000,22500.000000
max,96108.100000,73126.950000,102500.000000


We can see that, in general, fraudulent transactions have slightly more variabilities in the data, slightly less credit and credit limit. Fraudulent transactions also have slightly higher transaction amount for most transactions, but lacks big transactions. 
</br>

For categorical columns USER_AGENT and CITY, we can find how many unique catagories are there for these two columns: 

In [9]:
print('There are {} unique USER AGENT string'.format(len(set(data['USER_AGENT']))))
print('There are {} unique city'.format(len(set(data['CITY']))))

There are 1230 unique USER AGENT string
There are 916 unique city


</br>

For flag columns, we can look at if there is any difference between fraudulent and non-fraudulent transactions for positive flags.

In [10]:
flagdf = data.iloc[:,np.r_[4,11:37]]
flagfraudcount = flagdf.groupby(['FRAUD_FLAG']).sum()
flagfraudperc = flagfraudcount/flagfraudcount.sum()
flagfraudperc

,FLAG_LX,FLAG_ATM,FLAG_AUTO,FLAG_CASH,FLAG_LS,FLAG_DISCOUNT,FLAG_RECREA,FLAG_ELCTRNCS,FLAG_REG_AMT,FLAG_FASTFOOD,...,FLAG_LOW_AMT,FLAG_MANUAL_ENTRY,FLAG_PHONE_ORDER,FLAG_PURCHASE_EXCLUDING_GAS,FLAG_PLANNED,FLAG_RISKY,FLAG_SWIPE,FLAG_TRAVEL_ONLY,FLAG_TRAVEL_AND_ENTERTAINMENT,FLAG_WEEKEND
FRAUD_FLAG,,,,,,,,,,,,,,,,,,,,,
0.0,0.982287,0.872973,0.963888,0.868613,0.97373,0.960879,0.975355,0.921209,0.96618,0.989193,...,0.983893,0.979076,0.95135,0.974947,0.980316,0.875566,0.984823,0.979832,0.984437,0.974195
1.0,0.017713,0.127027,0.036112,0.131387,0.02627,0.039121,0.024645,0.078791,0.03382,0.010807,...,0.016107,0.020924,0.04865,0.025053,0.019684,0.124434,0.015177,0.020168,0.015563,0.025805


In [11]:
flagpercfilter = flagfraudperc.iloc[[1]][flagfraudperc.iloc[[1]].gt(0.1) | flagfraudperc.iloc[[1]].lt(0.01)]
flagpercfilter[flagpercfilter.columns[~flagpercfilter.isnull().all()]]

,FLAG_ATM,FLAG_CASH,FLAG_INTERNATIONAL,FLAG_RISKY
FRAUD_FLAG,,,,
1.0,0.127027,0.131387,0.255484,0.124434


The overall ratio of non-fraud transactions vs. fraud is 97.6% vs. 2.4%, here we see that some flag columns have some large deviation from this percentage. Fraudulent transactions takes up more than 10% of all ATM, cash, international transactions, and transactions marked as risky. There is no flag that has especially small percentage of fradulent transactions. 


</br>
</br>

For the rest of the numerical values, we compared the mean of fraud transaction vs. non fraud transaction through t.test. This step is done in R for the simplicity of t.test() function in R. we show the result here for columns with p.value > 0.05 in the t.test, ordered smallest to largest. 

In [12]:
Columns_to_Look = pd.read_csv(r"C:\Users\\Downloads\Winter 2023 Discovery Days\Columns_to_Look.csv")
Columns_to_Look

,COUNT_FASTFOOD_PAST_30DAY,COUNT_SWIPE_PAST_30DAY,COUNT_PLANNED_PAST_30DAY,COUNT_GAS_PAST_30DAY,SUM_FASTFOOD_PAST_30DAY,STD_LOW_AMT_PAST_30DAY,MEAN_LOW_AMT_PAST_30DAY,COUNT_TRAVEL_AND_ENTERTAINMENT_PAST_30DAY,COUNT_HIGH_AMT_PAST_30DAY,COUNT_PHONE_ORDER_PAST_30DAY,...,COUNT_LOW_AMT_PAST_7DAY,PREV_M_INFLATION,MEAN_WEEKEND_PAST_30DAY,MAX_RISKY_PAST_30DAY,STD_ELCTRNCS_PAST_30DAY,COUNT_WEEKEND_PAST_7DAY,SUM_ELCTRNCS_PAST_30DAY,STD_PARTIAL_PAST_30DAY,STD_PHONE_ORDER_PAST_30DAY,PREV_M_UNEMP_RATE
0,0,0,4.708847e-317,0.0,2.760000e-257,1.010000e-233,2.290000e-136,5.530000e-124,2.540000e-105,1.000000e-89,...,0.00414,0.005518,0.006253,0.011775,0.012316,0.015116,0.020036,0.020838,0.022265,0.040249


In [14]:
Columns_to_Look.iloc[:, : 10].min()

COUNT_FASTFOOD_PAST_30DAY                     0.000000e+00
COUNT_SWIPE_PAST_30DAY                        0.000000e+00
COUNT_PLANNED_PAST_30DAY                     4.708847e-317
COUNT_GAS_PAST_30DAY                          0.000000e+00
SUM_FASTFOOD_PAST_30DAY                      2.760000e-257
STD_LOW_AMT_PAST_30DAY                       1.010000e-233
MEAN_LOW_AMT_PAST_30DAY                      2.290000e-136
COUNT_TRAVEL_AND_ENTERTAINMENT_PAST_30DAY    5.530000e-124
COUNT_HIGH_AMT_PAST_30DAY                    2.540000e-105
COUNT_PHONE_ORDER_PAST_30DAY                  1.000000e-89
dtype: float64

We can see that the difference between fraud transactions and non fraud transactions is big between many attributes associated with information about the past 30 days. This insight can be beneficial when we are running the model. 